In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
os.listdir('../input/cassava-leaf-disease-classification/')

['train_tfrecords',
 'sample_submission.csv',
 'test_tfrecords',
 'label_num_to_disease_map.json',
 'train_images',
 'train.csv',
 'test_images']

In [3]:
train = pd.read_csv('../input/cassava-leaf-disease-merged/merged.csv')
test = pd.read_csv('../input/cassava-leaf-disease-classification//sample_submission.csv')
label_map = pd.read_json('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json', orient='index')

## Directory settings

In [4]:
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    
TRAIN_PATH = '../input/cassava-leaf-disease-merged/train'
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'

## CFG

In [5]:
class CFG:
    debug = False
    apex = False
    print_freq = 100
    num_workers = 4
    model_name = '1D_CNN'
    size = 256
    scheduler = 'CosineAnnealingWarmRestarts'
    epochs = 30
    T_0 = 10
    lr = 1e-3
    min_lr = 1e-6
    batch_size = 256
    weight_decay = 1e-6
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    seed = 42
    target_size = 5
    target_col = 'label'
    n_fold = 5
    trn_fold = [0, 1, 2, 3, 4]
    train = True
    inference = False
    
if CFG.debug:
    CFG.epochs = 1
    train = train.sample(n=1000, random_state=CFG.seed).reset_index(drop=True)

## Library

In [6]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm

import warnings 
warnings.filterwarnings('ignore')

if CFG.apex:
    from apex import amp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Utils

In [7]:
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f}')
    
def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

## CV split

In [8]:
oof_B3ns = pd.read_csv('../input/041-create-oof-efficientnet-b3ns-data/oof_df_0.csv')
oof_se = pd.read_csv('../input/nb023-data/oof_df.csv')

oof_B3ns_ = oof_B3ns[['0', '1', '2', '3', '4']]
oof_se_ = oof_se[['0', '1', '2', '3', '4']]
oof_se_.columns = ['5', '6', '7', '8', '9']

train = pd.concat([oof_B3ns_, oof_se_, oof_se[['label']]], axis=1)
train

,0,1,2,3,4,5,6,7,8,9,label
0,3.826102e-15,4.340454e-16,5.234761e-11,1.000000e+00,4.762953e-14,1.522047e-14,9.014521e-16,1.139739e-09,1.000000e+00,3.116005e-13,3
1,9.538939e-01,1.037140e-02,1.156314e-06,8.508536e-07,3.573275e-02,9.599438e-01,7.439279e-03,1.127475e-06,8.668555e-07,3.261490e-02,4
2,4.627016e-18,1.426970e-14,1.898384e-14,1.000000e+00,4.604767e-15,3.705072e-17,5.054389e-14,2.114423e-13,1.000000e+00,2.999360e-14,3
3,4.606794e-07,1.124587e-05,1.638323e-02,9.827052e-01,8.998843e-04,3.833728e-07,9.185316e-06,1.194510e-02,9.875053e-01,5.400496e-04,3
4,6.382671e-06,1.690052e-05,4.985041e-04,1.611800e-05,9.994621e-01,4.209951e-06,7.615774e-06,2.362547e-04,8.609924e-06,9.997434e-01,2
...,...,...,...,...,...,...,...,...,...,...,...
26332,8.475993e-07,9.982687e-01,2.881717e-07,1.730106e-03,8.013854e-08,9.047382e-07,9.980873e-01,3.099606e-07,1.911314e-03,6.628071e-08,1
26333,4.107651e-02,9.103698e-01,1.831035e-02,2.917714e-03,2.732570e-02,2.457832e-02,9.492393e-01,9.367346e-03,1.033589e-03,1.578141e-02,1
26334,3.930135e-09,1.206950e-05,3.702277e-11,9.999880e-01,3.137060e-13,6.578338e-09,1.752745e-05,4.747458e-11,9.999825e-01,7.934944e-13,3
26335,3.089139e-07,2.266481e-06,4.496734e-04,9.995383e-01,9.492393e-06,1.146029e-07,8.421153e-07,5.965448e-04,9.994010e-01,1.521211e-06,3


In [9]:
folds = train.copy()
Fold = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds[CFG.target_col])):
    folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)
print(folds.groupby(['fold', CFG.target_col]).size())

fold  label
0     0         299
      1         695
      2         603
      3        3093
      4         578
1     0         299
      1         695
      2         603
      3        3093
      4         578
2     0         298
      1         695
      2         604
      3        3092
      4         578
3     0         298
      1         695
      2         604
      3        3092
      4         578
4     0         298
      1         696
      2         603
      3        3092
      4         578
dtype: int64


In [10]:
folds

,0,1,2,3,4,5,6,7,8,9,label,fold
0,3.826102e-15,4.340454e-16,5.234761e-11,1.000000e+00,4.762953e-14,1.522047e-14,9.014521e-16,1.139739e-09,1.000000e+00,3.116005e-13,3,0
1,9.538939e-01,1.037140e-02,1.156314e-06,8.508536e-07,3.573275e-02,9.599438e-01,7.439279e-03,1.127475e-06,8.668555e-07,3.261490e-02,4,2
2,4.627016e-18,1.426970e-14,1.898384e-14,1.000000e+00,4.604767e-15,3.705072e-17,5.054389e-14,2.114423e-13,1.000000e+00,2.999360e-14,3,2
3,4.606794e-07,1.124587e-05,1.638323e-02,9.827052e-01,8.998843e-04,3.833728e-07,9.185316e-06,1.194510e-02,9.875053e-01,5.400496e-04,3,2
4,6.382671e-06,1.690052e-05,4.985041e-04,1.611800e-05,9.994621e-01,4.209951e-06,7.615774e-06,2.362547e-04,8.609924e-06,9.997434e-01,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
26332,8.475993e-07,9.982687e-01,2.881717e-07,1.730106e-03,8.013854e-08,9.047382e-07,9.980873e-01,3.099606e-07,1.911314e-03,6.628071e-08,1,4
26333,4.107651e-02,9.103698e-01,1.831035e-02,2.917714e-03,2.732570e-02,2.457832e-02,9.492393e-01,9.367346e-03,1.033589e-03,1.578141e-02,1,3
26334,3.930135e-09,1.206950e-05,3.702277e-11,9.999880e-01,3.137060e-13,6.578338e-09,1.752745e-05,4.747458e-11,9.999825e-01,7.934944e-13,3,0
26335,3.089139e-07,2.266481e-06,4.496734e-04,9.995383e-01,9.492393e-06,1.146029e-07,8.421153e-07,5.965448e-04,9.994010e-01,1.521211e-06,3,2


In [11]:
train = np.stack([df.values for df in [oof_B3ns_, oof_se_]], axis=2)
print(train.shape)

(26337, 5, 2)


## Dataset

In [12]:
class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.labels = df['label'].values
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TRAIN_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        label = torch.tensor(self.labels[idx]).long()
        return image, label
    
class StackingDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx]).float()
        y = torch.tensor(self.y[idx]).long()
        return X, y
    
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [13]:
train_dataset = StackingDataset(train, folds['label'])

for i in range(1):
    X, y = train_dataset[i]
    print(X.size())

torch.Size([5, 2])


## Transforms

In [14]:
def get_transforms(*, data):
    
    if data == 'train':
        return Compose([
            RandomResizedCrop(CFG.size, CFG.size), 
            Transpose(p=0.5), 
            HorizontalFlip(p=0.5), 
            VerticalFlip(p=0.5), 
            ShiftScaleRotate(p=0.5), 
            Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
            ), 
            ToTensorV2(),
        ])
    
    elif data == 'valid':
        return Compose([
            Resize(CFG.size, CFG.size), 
            Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
            ), 
            ToTensorV2(),
        ])

In [15]:
# train_dataset = StackingDataset(train, transform=get_transforms(data='train'))

# for i in range(1):
#     image, label = train_dataset[i]
#     plt.imshow(image[0])
#     plt.title(f'label: {label}')
#     plt.show()

## MODEL

In [16]:
class CustomResNext(nn.Module):
    def __init__(self, model_name='resnext50_32x4d', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, CFG.target_size)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [17]:
class StackingModel(nn.Module):
    def __init__(self, num_features, num_targets, dropout_ratio):
        super(StackingModel, self).__init__()
        
        self.conv1d_1 = nn.Conv1d(in_channels=num_features, 
                                  out_channels=10, 
                                  kernel_size=2, 
                                  stride=1, 
                                  padding=0)
        self.relu_1 = nn.ReLU()
        
        self.linear_2 = nn.Linear(10, 16)
        self.relu_2 = nn.ReLU()
        self.dropout_2 = nn.Dropout(dropout_ratio)
        
        self.linear_3 = nn.Linear(16, num_targets)
        
    def forward(self, x, batch_size):
        x = self.conv1d_1(x)
        x = self.relu_1(x)
        x = torch.reshape(x, (batch_size, -1))
        
        x = self.linear_2(x)
        x = self.relu_2(x)
        x = self.dropout_2(x)
        
        x = self.linear_3(x)
        
        return x

In [18]:
model = StackingModel(num_features=CFG.target_size, num_targets=CFG.target_size, dropout_ratio=0.2)
train_dataset = StackingDataset(train, folds['label'])
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, 
                          num_workers=4, pin_memory=True, drop_last=True)

for X, y in train_loader:
    print(X.size())
    output = model(X, CFG.batch_size)
    print(output.size())
    break

torch.Size([256, 5, 2])
torch.Size([256, 5])


In [19]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def asMinutes(s):
    """秒を分に変換する関数"""
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    """経過時間の測定と終了時間の予測を行う関数
    Parameters
    ----------
    since : float
        実験を始めた時刻
    percent : float
        実験が進んだ割合
        
    Returns
    -------
    s : 経過時間
    re : 終了までの時間の予測
    """
    now = time.time()
    s = now - since  # 経過時間の測定
    es = s / percent  # 終了時間の予測
    re = es - s  # 残り時間の予想
    return '%s (remain %s)' % (asMinutes(s), asMinutes(re))

def train_fn(train_loader, model, criterion, optimizer, epoch, shechduler, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (X, y) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        X = X.to(device)
        y = y.to(device)
        batch_size = y.size(0)
        y_preds = model(X, batch_size)
        loss = criterion(y_preds, y)
        # record loss
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        if CFG.apex:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else: 
            loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}]'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})'
                  'Elapsed {remain:s}' 
                  'Loss: {loss.val:.4f}({loss.avg:.4f})' 
                  'Grad: {grad_norm:.4f}  '
                  .format(epoch+1, step, len(train_loader), batch_time=batch_time, 
                          data_time=data_time, loss=losses, 
                          remain=timeSince(start, float(step+1)/len(train_loader)), 
                          grad_norm=grad_norm))
    return losses.avg

def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    preds = []
    start = end = time.time()
    for step, (X, y) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        X = X.to(device)
        y = y.to(device)
        batch_size = y.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(X, batch_size)
        loss = criterion(y_preds, y)
        losses.update(loss.item(), batch_size)
        # record accuracy
        preds.append(y_preds.softmax(1).to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
            
    predictions = np.concatenate(preds)
    return losses.avg, predictions

def inference(model, states, test_loader, device):
    model.to(device)
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avgpreds = []
        for state in states:
            model.load_state_dict(state['model'])
            model.eval()
            with torch.no_grad():
                y_preds = model(images)
            avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

## Train loop

In [20]:
# ======================================================
# Train loop
# ======================================================

def train_loop(folds, fold):
    
    LOGGER.info(f'========== fold: {fold} training ============')
    
    # ======================================================
    # loader
    # ======================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index
    
    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)
    
    train_ = train[trn_idx]
    valid_ = train[val_idx]
    
    train_dataset = StackingDataset(train_, train_folds['label'])
    valid_dataset = StackingDataset(valid_, valid_folds['label'])
    
    train_loader = DataLoader(train_dataset, 
                              batch_size=CFG.batch_size, 
                              shuffle=True, 
                              num_workers=CFG.num_workers, 
                              pin_memory=True, 
                              drop_last=False)
    valid_loader = DataLoader(valid_dataset, 
                              batch_size=CFG.batch_size, 
                              shuffle=False, 
                              num_workers=CFG.num_workers, 
                              pin_memory=True, 
                              drop_last=False)
    
    # ===============================================
    # scheduler
    # ===============================================
    def get_scheduler(optimizer):
        if CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
        return scheduler
    
    # ===============================================
    # model & optimizer
    # ===============================================
    model = StackingModel(num_features=CFG.target_size, num_targets=CFG.target_size, dropout_ratio=0.5)
    model.to(device)
    
    optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = get_scheduler(optimizer)
    
    # ===============================================
    # apex 
    # ===============================================
    if CFG.apex:
        model.optimizer = amp.initialize(model, optimizer, opt_level='O1', verbosity=0)
        
    # ===============================================
    # loop
    # ===============================================
    criterion = nn.CrossEntropyLoss()
    
    best_score = 0.
    best_loss = np.inf
    
    for epoch in range(CFG.epochs):
        
        start_time = time.time()
        
        # train
        avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device)
        
        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        valid_labels = valid_folds[CFG.target_col].values
        
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, CosineAnnealingWarmRestarts):
            scheduler.step()
        
        # scoring
        score = get_score(valid_labels, preds.argmax(1))
        
        elapsed = time.time() - start_time
        
        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f} avg_val_loss: {avg_val_loss:.4f} time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Accuracy: {score}')
        
        if score > best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Mpdel')
            torch.save({'model': model.state_dict(), 
                        'preds': preds}, 
                        OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')
    check_point = torch.load(OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')
    valid_folds[[str(c) for c in range(5)]] = check_point['preds']
    valid_folds['preds'] = check_point['preds'].argmax(1)
    
    return valid_folds

In [21]:
# ====================================================
# main
# ====================================================
def main():
    
    """
    Prepare: 1.train 2.test 3.submission 4.folds
    """
    
    def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df[CFG.target_col].values
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.5f}')
        
    if CFG.train:
        # train
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(folds, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f'=============== fold: {fold} result ================')
                get_result(_oof_df)
        # CV result
        LOGGER.info(f'============ CV ============')
        get_result(oof_df)
        # save result
        oof_df.to_csv(OUTPUT_DIR+'oof_df.csv', index=False)
        
    if CFG.inference:
        # inference
        model = CustomResNext(CFG.model_name, pretrained=False)
        states = [torch.load(OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth') for fold in CFG.trn_fold]
        test_dataset = TestDataset(test, batch_size=CFG.batch_size, shuffle=False, pin_memory=True)
        predictions = inference(model, states, test_loader, device)
        # submission
        test['label'] = predictions.argmax(1)
        test[['image_id', 'label']].to_csv(OUTPUT_DIR+'submission.csv', index=False)

In [22]:
if __name__ == '__main__':
    main()

========== fold: 0 training ============


Epoch: [1][0/83]Data 0.104 (0.104)Elapsed 0m 0s (remain 0m 13s)Loss: 1.6677(1.6677)Grad: 0.6472  
Epoch: [1][82/83]Data 0.000 (0.004)Elapsed 0m 0s (remain 0m 0s)Loss: 1.4484(1.5518)Grad: 0.4470  
EVAL: [0/21] Data 0.104 (0.104) Elapsed 0m 0s (remain 0m 2s) Loss: 1.4166(1.4166) 


Epoch 1 - avg_train_loss: 1.5518 avg_val_loss: 1.4122 time: 1s
Epoch 1 - Accuracy: 0.587129840546697
Epoch 1 - Save Best Score: 0.5871 Mpdel


EVAL: [20/21] Data 0.000 (0.010) Elapsed 0m 0s (remain 0m 0s) Loss: 1.4758(1.4122) 
Epoch: [2][0/83]Data 0.102 (0.102)Elapsed 0m 0s (remain 0m 9s)Loss: 1.4004(1.4004)Grad: 0.5594  
Epoch: [2][82/83]Data 0.000 (0.004)Elapsed 0m 0s (remain 0m 0s)Loss: 1.1552(1.2272)Grad: 0.3798  
EVAL: [0/21] Data 0.112 (0.112) Elapsed 0m 0s (remain 0m 2s) Loss: 0.9383(0.9383) 


Epoch 2 - avg_train_loss: 1.2272 avg_val_loss: 0.9474 time: 1s
Epoch 2 - Accuracy: 0.6989369779802581
Epoch 2 - Save Best Score: 0.6989 Mpdel


...

EVAL: [20/21] Data 0.004 (0.009) Elapsed 0m 0s (remain 0m 0s) Loss: 0.4199(0.4010) 
Epoch: [29][0/83]Data 0.120 (0.120)Elapsed 0m 0s (remain 0m 10s)Loss: 0.5650(0.5650)Grad: 0.1988  
Epoch: [29][82/83]Data 0.000 (0.004)Elapsed 0m 0s (remain 0m 0s)Loss: 0.3876(0.5202)Grad: 0.3277  
EVAL: [0/21] Data 0.119 (0.119) Elapsed 0m 0s (remain 0m 2s) Loss: 0.3485(0.3485) 


Epoch 29 - avg_train_loss: 0.5202 avg_val_loss: 0.4007 time: 1s
Epoch 29 - Accuracy: 0.8904499715207899


EVAL: [20/21] Data 0.000 (0.009) Elapsed 0m 0s (remain 0m 0s) Loss: 0.4196(0.4007) 
Epoch: [30][0/83]Data 0.092 (0.092)Elapsed 0m 0s (remain 0m 8s)Loss: 0.4293(0.4293)Grad: 0.1646  
Epoch: [30][82/83]Data 0.000 (0.004)Elapsed 0m 0s (remain 0m 0s)Loss: 0.4186(0.5240)Grad: 0.2592  
EVAL: [0/21] Data 0.093 (0.093) Elapsed 0m 0s (remain 0m 1s) Loss: 0.3485(0.3485) 


Epoch 30 - avg_train_loss: 0.5240 avg_val_loss: 0.4006 time: 1s
Epoch 30 - Accuracy: 0.8904499715207899
=============== fold: 4 result ================
Score: 0.89102
============ CV ============
Score: 0.89376


EVAL: [20/21] Data 0.002 (0.008) Elapsed 0m 0s (remain 0m 0s) Loss: 0.4195(0.4006) 
